In [3]:
!pip install dask



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import dask.dataframe as dd

# Example of reading CSV
final = dd.read_csv('merged_final4.csv', assume_missing=True)

# Optional: check columns
print(final.columns)  # should show 'primaryName'


Index(['unnamed_0', 'tconst', 'primaryTitle', 'runtimeMinutes', 'genres',
       '_lang', 'language_name', '_lang_right', 'nconst', 'ordering',
       'category', 'unnamed_0_right', 'primaryName', 'knownForTitles',
       'primaryProfession', 'unnamed_0_right_right', 'averageRating'],
      dtype='object')


In [15]:
import glob
import polars as pl
import os

# ✅ Path to your folder containing the .part files
path = r"C:\Users\Amanjot Singh\Desktop\movie project\updated_language.csv"

# Find all .part files inside the folder
files = glob.glob(os.path.join(path, "*.part"))
print("Found files:", files)

if not files:
    print("⚠️ No .part files found! Check if this path is a folder (not a single .csv file).")
else:
    # Sort and read all parts efficiently
    df = pl.concat([pl.read_csv(f) for f in sorted(files)])
    
    # Save to one merged file
    output_path = r"C:\Users\Amanjot Singh\Desktop\movie project\final_updated_language.csv"
    df.write_csv(output_path)
    
    print(f"✅ Merged {len(files)} parts successfully into:\n{output_path}")


Found files: ['C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\00.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\01.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\02.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\03.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\04.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\05.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\06.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\07.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\08.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\09.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\10.part', 'C:\\Users\\Amanjot Singh\\Desktop\\movie project\\updated_language.csv\\

In [2]:
pip install polars

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached polars-1.34.0-py3-none-any.whl.metadata (10 kB)
  Using cached polars_runtime_32-1.34.0-cp39-abi3-win_amd64.whl.metadata (1.5 kB)
Using cached polars-1.34.0-py3-none-any.whl (772 kB)
Using cached polars_runtime_32-1.34.0-cp39-abi3-win_amd64.whl (40.1 MB)

   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
   -------------------- ------------------- 1/2 [polars]
 

In [1]:
import polars as pl

# 🔹 Read both CSVs lazily
df1 = pl.scan_csv(r"C:\Users\Amanjot Singh\Desktop\movie project\merged_final3.csv")
df2 = pl.scan_csv(r"C:\Users\Amanjot Singh\Desktop\movie project\updated_rating.csv")

# ⚙️ Clean both dataframes before join
def clean_columns(df: pl.LazyFrame):
    cols = [c if c.strip() != "" else f"unnamed_{i}" for i, c in enumerate(df.columns)]
    df = df.rename(dict(zip(df.columns, cols)))
    # remove duplicate column names by adding suffixes
    seen = {}
    new_cols = []
    for c in df.columns:
        if c in seen:
            seen[c] += 1
            new_cols.append(f"{c}_{seen[c]}")
        else:
            seen[c] = 0
            new_cols.append(c)
    df = df.rename(dict(zip(df.columns, new_cols)))
    return df

df1 = clean_columns(df1)
df2 = clean_columns(df2)

# 🔹 Rename overlapping columns in df2 (except join key)
common_cols = [c for c in df2.columns if c in df1.columns and c != "tconst"]
df2 = df2.rename({c: f"{c}_right" for c in common_cols})

# 🔹 Join safely
merged_lazy = df1.join(df2, on="tconst", how="inner")

# 🔹 Collect and remove duplicates
merged_df = merged_lazy.collect().unique()

# 🔹 Save final merged dataset
merged_df.write_csv(r"C:\Users\Amanjot Singh\Desktop\movie project\merged_final4.csv")

print("✅ Merge complete, duplicates handled, and saved as merged_final4.csv")


C:\Users\Amanjot Singh\AppData\Local\Temp\ipykernel_15048\3922497480.py:9: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  cols = [c if c.strip() != "" else f"unnamed_{i}" for i, c in enumerate(df.columns)]
C:\Users\Amanjot Singh\AppData\Local\Temp\ipykernel_15048\3922497480.py:10: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df = df.rename(dict(zip(df.columns, cols)))
C:\Users\Amanjot Singh\AppData\Local\Temp\ipykernel_15048\3922497480.py:14: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get th

✅ Merge complete, duplicates handled, and saved as merged_final4.csv


In [3]:
import polars as pl
l = pl.scan_csv('merged_final3.csv')
l.columns

C:\Users\Amanjot Singh\AppData\Local\Temp\ipykernel_13736\3954128216.py:3: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  l.columns


['unnamed_0',
 'tconst',
 'primaryTitle',
 'runtimeMinutes',
 'genres',
 '_lang',
 'language_name',
 '_lang_right',
 'nconst',
 'ordering',
 'category',
 'unnamed_0_right',
 'primaryName',
 'knownForTitles',
 'primaryProfession']